In [1]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state=42
)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, random_state=42
)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [16]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [18]:
from sklearn.model_selection import cross_validate
scores= cross_validate(dt, train_input, train_target)
display(scores)

{'fit_time': array([0.01800418, 0.01015759, 0.01063228, 0.01121187, 0.01007915]),
 'score_time': array([0.00118709, 0.00115681, 0.00117087, 0.0011487 , 0.0011878 ]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [20]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [21]:
from sklearn.model_selection import StratifiedKFold
scores= cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [30]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8703105083740921


In [25]:
# cv -> cross validate (교차검증)

from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [26]:
# 그리드서치 객체화
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)

In [27]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [28]:
# params 중에 가장 최고 값
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [31]:
# params 중 어떤 paramater가 최고값?
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [32]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [33]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [39]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [40]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [41]:
from scipy.stats import uniform, randint

In [44]:
rgen = randint(0,10)
rgen.rvs(10)


array([1, 5, 0, 6, 4, 7, 3, 0, 9, 3])

In [45]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([108,  97,  98, 112,  79,  93, 115,  94, 108,  96]))

In [46]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.08587875, 0.73532589, 0.74676219, 0.73605255, 0.05151934,
       0.48768276, 0.80284216, 0.39756689, 0.36322841, 0.31384131])

In [47]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),}

In [55]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7efd9eb1dc10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7efd9eb05390>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7efd9e99a310>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7efd9eb1d8d0>},
                   random_state=42)

In [56]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [57]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [51]:
dt=gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
